#### Custom Root Types

Pydantic models can be defined with a custom root type by declaring the `__root__` field.

The root type can be any type supported by pydantic, and is specified by the type hint on the `__root__` field. The root value can be passed to the model `__init__` via the `__root__` keyword argument, or as the first and only argument to `parse_obj`.

In [1]:
import json
from typing import List, Dict
from pydantic.schema import schema
from pydantic import BaseModel, ValidationError

In [2]:
class Pets(BaseModel):
    __root__: List[str]

In [3]:
print(Pets(__root__=["dog", "cat"]))

__root__=['dog', 'cat']


In [4]:
print(Pets(__root__=["dog", "cat"]).json())

["dog", "cat"]


In [5]:
print(Pets.parse_obj(["dog", "cat"]))

__root__=['dog', 'cat']


In [6]:
print(Pets.schema())

{'title': 'Pets', 'type': 'array', 'items': {'type': 'string'}}


In [7]:
pets_schema = schema([Pets])
print(json.dumps(pets_schema, indent=4))

{
    "definitions": {
        "Pets": {
            "title": "Pets",
            "type": "array",
            "items": {
                "type": "string"
            }
        }
    }
}


If you call the `parse_obj` method for a model with a custom root type with a dict as the first argument, the following logic is used:

* If the custom root type is a mapping type (eg., `Dict` or `Mapping`), the argument itself is always validated against the custom root type.

* For other custom root types, if the dict has precisely one key with the value `__root__`, the corresponding value will be validated against the custom root type.

* Otherwise, the dict itself is validated against the custom root type.

In [8]:
print(Pets.parse_obj(["dog", "cat"]))

__root__=['dog', 'cat']


In [9]:
print(Pets.parse_obj({"__root__": ["dog", "cat"]}))

__root__=['dog', 'cat']


In [10]:
class PetsByName(BaseModel):
    __root__: Dict[str, str]

In [11]:
print(PetsByName.parse_obj({"Otis": "dog", "Milo": "cat"}))

__root__={'Otis': 'dog', 'Milo': 'cat'}


In [12]:
try:
    print(PetsByName.parse_obj({"__root__": {"Otis": "dog", "Milo": "cat"}}))
except ValidationError as e:
    print(e)

1 validation error for PetsByName
__root__ -> __root__
  str type expected (type=type_error.str)


> ##### Warning
> 
> Calling the `parse_obj` method on a dict with the single key `"__root__"` for non-mapping custom root types is currently supported for backwards compatibility, but is not recommended and may be dropped in a future version.

If you want to access items in the `__root__` field directly or to iterate over the items, you can implement custom `__iter__` and `__getitem__` functions, as shown in the following example.

In [13]:
class MappedPets(BaseModel):
    __root__: List[str]

    def __iter__(self):
        return iter(self.__root__)

    def __getitem__(self, item):
        return self.__root__[item]

In [14]:
pets = MappedPets.parse_obj(["dog", "cat"])
print(pets[0])

dog


In [15]:
print([pet for pet in pets])

['dog', 'cat']
